# Libraries

In [ ]:
import os
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
from rasterio.enums import Resampling
from datetime import datetime, timedelta
from calendar import monthrange

In [14]:
# Set the base directory with an environment variable, fallback to a relative path
main_dir = os.getenv('PROJECT_DIR', os.path.join(os.getcwd(), 'data', 'ANIN', 'Generating Indices', 'VCI', 'NDVI-16', 'Trial', 'Tiles'))
# Directory to years files for MODIS data of AOI
MODIS = os.path.join(main_dir, 'Clipped_trial') 
# Directory to the shape file of the three countries
Boundary = os.path.join(main_dir, 'Boundary')
# Path to the image to be used as a reference crs
Copernicus_ex_path = os.getenv('PROJECT_DIR', os.path.join(os.getcwd(), 'data', 'ANIN', 'Generating Indices', 'VCI', 'NDVI-16', 'Trial', 'Tiles', 'Copernicus_monthly', '2020', '2020August.tif'))
# open the image as xarray
Copernicus_ex = rxr.open_rasterio(Copernicus_ex_path, masked=True)
# get the image's crs
Copernicus_crs = Copernicus_ex.rio.crs

# Monthly MODIS from 16Days Composite

The methodology to calculate monthly NDVI data of Modis is different from that for Copernicus data.
The 250m NDVI data of Modis are every 16 days, and there are some NDVI data which overlaps two months, 
for example, the 16 days starts in January and ends in February.
Therefore, the monthly NDVI data will be calculated using the weighted average based on the number of days in a month

In [3]:
'''
The day of the year of the start of the 16 days is extracted from 
the name of NDVI file. 
There are two functions, one if the start data is in the month and the other
if the end data is in the month.
'''
#Function if the start date of NDVI is in the month.
def weights_start(Start_date):
    '''
    Inputs:
         date in the format (dd-mm-yyyy)
    returns:
        weight based on the number of observation days overlap the month
    '''
    #Calculation of the number of days on the month 
    num_days = monthrange(int(Start_date.strftime('%Y')),
                          int(Start_date.strftime('%m')))[1]
    # The starting day of the 16Days NDVI in the month
    day_of_month = int(Start_date.strftime('%d'))
    # Number of days bewteen the starting data and the end of the month
    diff = (num_days +1) - day_of_month
    #Ofcourse if the Number of days bewteen the starting data and the end of 
    #the month is greater than 16, that means that all 16 days are in the month
    if diff > 16:
        day_obs = 16 #day_obs: days of observation NDVI in the month
    #If the difference is less than 16 days that means the days of observation
    #in that month equal to the number of days bewteen the starting data and 
    #the end of the month.
    else: 
        day_obs = diff
    # Calculaion of the weight based on the days of observation NDVI
    # in the month
    weight = (day_obs / num_days)
    
    return weight
#Function if the end date of NDVI is in the month.
def weights_end(End_date):
    '''
    Inputs:
         date in the format (dd-mm-yyyy)
    returns:
        weight based on the number of observation days overlap the month
    '''
    #Calculation of the number of days on the month 
    num_days = monthrange(int(End_date.strftime('%Y')),
                          int(End_date.strftime('%m')))[1]
    # The ending day of the 16Days NDVI in the month
    day_of_month = int(End_date.strftime('%d'))
    # The days of of observation NDVI is equal to end date,
    # but it will be a further steps
    #in the following lines of code
    day_obs = day_of_month
    
    weight = (day_obs / num_days)
     # Calculaion of the weight based on the days of observation NDVI
     # in the month
    return weight

In [5]:
'''
The code is built based on that there are years folders which contain all
the 16days NDVI data in that year
'''
#list of all years to be used to calculate max and min NDVI
years = sorted(os.listdir(MODIS))
#Empty list to contain other lists of the monthly NDVI data for
#each month through years
#for example, list of monthly NDVI of January of each year
months_through_years = []
#looping through 12 months, the looping started by months not years 
#as the goal is to calculate histrocal max and min values for each month
for month in range(1,13):
    #Empty list for the monthly NDVI data for each month through years
    lis_of_m =[]
    #looping through years
    for year in years:
        #Empty list to contain all 16days NDVI overlapping in the month
        month_in_year = []
        #looping through the 16Days NDVI of the year
        for image in os.listdir(os.path.join(MODIS, year)):
            #Read the year from name of the file
            y = image[9:13]
            #Read the day of the year from the name of the file
            DOY = image[13:16]
            #convert the day of the year to a date in the format (DD-MM-yyyy)
            DOY.rjust(3 + len(DOY), '0')
            Date_of_start = datetime.strptime(y + "-" + DOY, "%Y-%j")
            Date_of_end = Date_of_start + timedelta(days=16)
            #If the starting date is in the month
            if int(Date_of_start.strftime('%m')) == month:
                #Apply the function to calculate the weight based on 
                #the starting date
                weight = weights_start(Date_of_start)
                #The path of the image
                image = os.path.join(MODIS, year, image)
                #Open the tiff file of 16Days NDVI
                NDVI = rxr.open_rasterio(image,
                                  masked=True)
                # Multiply by the scalling factor to get the real NDVI values
                NDVI = NDVI*0.0001
                #Multiply the 16Days NDVI with the weight 
                img_weighted = weight * NDVI
                #Append th weighted 16Days NDVI to the list of all images 
                #in that month
                month_in_year.append(img_weighted)
            #If the ending date is in the month  
            elif int(Date_of_end.strftime('%m')) == month:
                # If the end day is after 15th of the month,
                #that means the image was already considered in starting date,
                #so nothing will happen.
                if int(Date_of_end.strftime('%d')) >15:
                    continue
                #If the end day is in January, it should not be considered as
                #the first image of a year is always starts in 1st of January 
                if int(Date_of_end.strftime('%m')) == 1:
                    continue
                    
                #Apply the function to calculate the weight based on
                #the ending date
                weight = weights_end(Date_of_end)
                #The path of the image
                image = os.path.join(MODIS, year, image)
                #Open the tiff file of 16Days NDVI
                NDVI = rxr.open_rasterio(image,
                                  masked=True)
                # Multiply by the scalling factor to get the real NDVI values
                NDVI = NDVI * 0.0001
                 #Multiply the 16Days NDVI with the weight
                img_weighted = weight * NDVI
                #Append th weighted 16Days NDVI to the list of all images
                #in that month
                month_in_year.append(img_weighted)
              
        #Calculating the monthly composite by adding
        #the weighted averaged 16Days NDVI overlap this month
        mothly_composite = sum(month_in_year)
        #Append the monthly composite to the list of this month through years
        lis_of_m.append(mothly_composite)
    #Append lists of NDVI data of each month through years 
    months_through_years.append(lis_of_m)               
print('lists of NDVI data of each month through years is created')   

lists of NDVI data of each month through years is created


In [ ]:
#List of months to use them as keys for dictionaries
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
          'October', 'November', 'December']
#Empty dictionary which will contain keys of each month with the corresponding
#maximum NDVI as values
NDVI_max= {}
#Empty dictionary which will contain keys of each month with the corresponding
#minimum NDVI as values
NDVI_min= {}
#Index to be used to get the name of month from months list
count =0
#loop through the list of lists of monthly NDVI data of each month
for month in months_through_years:
    #Concatenation of all monthly NDVI data of the month through years
    #for example, NDVI data of January for whole years
    concat = xr.concat(month, dim='band')
    #Calculation of max NDVI for the month
    Max = concat.max(dim='band')
    #Resampling max NDVI from 250m to 300m as Copernicus NDVI data 
    #with the use of the same crs
    Max_AOI = Max.rio.reproject(Copernicus_crs,
                                shape = Copernicus_ex.rio.shape, 
                                resampling=Resampling.nearest).rio.reproject_match(Copernicus_ex)
    #Replace the nodata value which is 3.402823466e+38 t be NAN
    Max_AOI = Max_AOI.where(Max_AOI <1)    
    #Append to the dictionary as following: 
    #the key is the name of the month
    #the value is the max NDVI for the AOI
    NDVI_max[months[count]] =Max_AOI
            
    #The procedures of max NDVI is repeated for min NDVI
    
    Min = concat.min(dim='band')
    Min_AOI = Min.rio.reproject(Copernicus_crs,
                                shape = Copernicus_ex.rio.shape,
                                resampling=Resampling.nearest).rio.reproject_match(Copernicus_ex)
    Min_AOI = Min_AOI.where(Min_AOI <1)
    NDVI_min[months[count]] = Min_AOI
    
    #The following lines are optional if we need to export
    #max and min NDVI as tiff files  
    
    MAX = os.path.join(main_dir, ('MAX'))
    check_folder_MAX = os.path.isdir(MAX)
    if not check_folder_MAX:
        os.makedirs(MAX)
    #Export max NDVI as tiff file
    Max_AOI.rio.to_raster(os.path.join(MAX, months[count] + '_max'+ '.tif'))
    
    MIN = os.path.join(main_dir, ('MIN'))
    check_folder_MIN = os.path.isdir(MIN)
    if not check_folder_MIN:
        os.makedirs(MIN)
    #Export min NDVI as tiff file
    Min_AOI.rio.to_raster(os.path.join(MIN, months[count] + '_min'+'.tif'))
    
    count =count +1    
print('Dictionaries of max/min NDVI data of each month through years is created')   